In [1]:
import collections
import copy
import datetime
import itertools
import json
import math
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
G = cibblbibbl.CIBBL
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay

In [7]:
data = collections.defaultdict(list)
for Mu in G.matchups:
    for teamId, d0 in Mu.config["player"].items():
        for playerId, p in d0.items():
            Pl = player(str(playerId))
            prespp = Pl.prespp(Mu)
            spp = p.get("spp", 0)
            data[Pl].append((Mu, prespp, spp))

In [8]:
uncommon = collections.defaultdict(list)
for Pl in sorted(data):
    L = data[Pl]
    if not isinstance(Pl, (cibblbibbl.player.Player, cibblbibbl.player.RaisedDeadPlayer)):
        continue
    expprespp = 0
    if Pl.prev:
        if data.get(Pl.prev):
            prevlastd = data[Pl.prev][-1]
        else:
            prevlastd = [None, 0, 0]
        expprespp = sum(prevlastd[1:3]) * Pl.prevsppmul
    nameprint = False
    for i, (Mu, prespp, spp) in enumerate(L):
        if prespp != expprespp:
            if not nameprint:
                s = f'[{Pl.Id}, {Pl.status}] {Pl.name} [{Pl.team.Id}, {Pl.team.name}]'
                if Pl.prev:
                    s += f' (prev: [{Pl.prev.Id}] {Pl.prev.name})'
                print(s)
                nameprint = True
            print(f'{Mu.match.Id}, exp:{expprespp}, got:{prespp}')
            uncommon[Pl].append([Mu, expprespp, prespp])
        expprespp = prespp + spp       
    if nameprint:
        print()

[12349401, Retired] Kkrozh [908529, ICBM Warpspawns] (prev: [12295352] Kkrozh'sazhs)
4046041, exp:8, got:11

[12372276, Active] Zzak [908529, ICBM Warpspawns] (prev: [12295363] Zzak'tarh)
4046041, exp:17, got:31

[12372278, Active] Tarh [908529, ICBM Warpspawns] (prev: [12295363] Zzak'tarh)
4046041, exp:23, got:37

[12618231, Active] Gashokh [908529, ICBM Warpspawns] (prev: [12456141] Gashokh'vanderburth)
4055462, exp:22, got:28

[12618232, Active] Vanderburth [908529, ICBM Warpspawns] (prev: [12456141] Gashokh'vanderburth)
4055462, exp:22, got:28



In [5]:
Players = uncommon.keys()

In [6]:
# carry previous SPPs (reload above three after it runned)
for Pl in Players:
    Pl0 = Pl.prev
    if not Pl0:
        continue
    prevmatchups = tuple(
        cibblbibbl.matchup.sort_by_modified(
            Mu
            for S in Pl0._matchups.values()
            for Mu in S
        )
    )
    if not prevmatchups:
        continue
    prevperf = prevmatchups[-1].performance(Pl0)
    prevspp = prevperf.get("prespp", 0)
    prevspp += prevperf.get("spp", 0)
    prevspp = math.floor(prevspp * Pl.prevsppmul)
    if prevspp:
        adminspp = Pl.adminspp
        adminspp["0"] = prevspp
        Pl.adminspp = adminspp
    print(Pl.Id, Pl.name, prevspp)

    

Haseem Abdul Sabbar 5
Fayid Mohammed 3
Calas Typhon 10
Varesh Kaurambi 7
Abd Allah Bousaid 33
Oogie Boogie 18
Gaudi 32
Kkrozh 3
Sazks 3
Zzak 14
Tarh 14
Harkon Halvarson 33
Garalash 9
Gashokh 22
Vanderburth 22


In [10]:
#fix admin spps
for Pl, L in uncommon.items():
    for Mu, expprespp, prespp in L:
        Ma = Mu.match
        value = expprespp - prespp
        adminspp = Pl.adminspp
        adminspp[str(Ma.Id)] = value
        print(Pl.Id, Pl.name, Ma.Id, value)

12349401 Kkrozh 4046041 -3
12372276 Zzak 4046041 -14
12372278 Tarh 4046041 -14
12618231 Gashokh 4055462 -6
12618232 Vanderburth 4055462 -6


In [8]:
Players = [player(12618231), player(12618232)]

In [11]:
Pl = Players[0]

In [7]:
Pl = player(12618231)
uncommon[Pl]

NameError: name 'uncommon' is not defined

In [12]:
Mu = Pl.matchups("cibbl")[0]

In [13]:
Pl.prespp(Mu)

22

In [14]:
Pl0 = Pl.prev

In [15]:
prevmatchups = tuple(
    cibblbibbl.matchup.sort_by_modified(
        Mu
        for S in Pl0._matchups.values()
        for Mu in S
    )
)

In [16]:
if prevmatchups:
    prevspp = prevmatchups[-1].performance(Pl0).get("prespp", 0)
else:
    prevspp = 0
prevspp

20

In [19]:
if prevspp:
    adminspp = Pl.adminspp
    adminspp["0"] = prevspp
    Pl.adminspp = adminspp

NormalPlayer('12138861') adminspp {'0': 32}


In [17]:
Pl.adminspp

{'0': 32}

In [18]:
Pl.config["adminspp"] = adminspp

In [14]:
{'0': 32} == {}

False

In [24]:
default = Pl.__class__.adminspp.default(Pl)

In [30]:
a = Pl.__class__.adminspp.__get__(Pl, Pl.__class__)
a

{'0': 32}

In [31]:
a == default

True

In [32]:
a is default

False

In [34]:
default

{}

In [35]:
type(a)

cibblbibbl.jsonfile.JSONFileObject

In [36]:
type(default)

dict

In [39]:
a== default

True

In [41]:
hash(a)

TypeError: unhashable type: 'JSONFileObject'

In [ ]:
[11744537, Active] Haseem Abdul Sabbar [873245, Sphinx Legion of Numas] (prev: [11736871] Haseem Abdul Sabbar)
3897789, exp:5, got:0

[11765469, Dead] Fayid Mohammed [873245, Sphinx Legion of Numas] (prev: [11736876] Fayid Mohammed)
3900603, exp:3, got:0

[11791157, Dead] Calas Typhon [871787, Malignant Curators] (prev: [11558607] Calas Typhon)
3905436, exp:10, got:0

[11940785, Retired] Varesh Kaurambi [871787, Malignant Curators] (prev: [11770088] Varesh Kaurambi)
3930641, exp:7, got:0

[11956674, Retired] Abd Allah Bousaid [871787, Malignant Curators] (prev: [11455996] Abd Allah Bousaid)
3933541, exp:33, got:0

[11978228, Retired] Oogie Boogie [882382, Ali's Anarchists] (prev: [11881505] Oogie Boogie)
3934598, exp:18, got:0

[12138861, Dead] Gaudi [882382, Ali's Anarchists] (prev: [11881471] Gaudi)
3964222, exp:32, got:0

[12349401, Retired] Kkrozh [908529, ICBM Warpspawns] (prev: [12295352] Kkrozh'sazhs)
4005344, exp:3, got:0
4046041, exp:5, got:8

[12349402, Retired] Sazks [908529, ICBM Warpspawns] (prev: [12295352] Kkrozh'sazhs)
4005344, exp:3, got:0

[12372276, Active] Zzak [908529, ICBM Warpspawns] (prev: [12295363] Zzak'tarh)
4010498, exp:14, got:0
4046041, exp:3, got:17

[12372278, Active] Tarh [908529, ICBM Warpspawns] (prev: [12295363] Zzak'tarh)
4010498, exp:14, got:0
4046041, exp:9, got:23

[12423015, Active] Harkon Halvarson [871787, Malignant Curators] (prev: [11892408] Harkon Halvarson)
4017192, exp:33, got:0

[12467315, Retired] Garalash [871787, Malignant Curators] (prev: [12103185] Garalash)
4023916, exp:9, got:0

[12618231, Active] Gashokh [908529, ICBM Warpspawns] (prev: [12456141] Gashokh'vanderburth)
4055462, exp:22, got:6

[12618232, Active] Vanderburth [908529, ICBM Warpspawns] (prev: [12456141] Gashokh'vanderburth)
4055462, exp:22, got:6



In [ ]:
[12349401, Retired] Kkrozh [908529, ICBM Warpspawns] (prev: [12295352] Kkrozh'sazhs)
4046041, exp:8, got:11

[12372276, Active] Zzak [908529, ICBM Warpspawns] (prev: [12295363] Zzak'tarh)
4046041, exp:17, got:31

[12372278, Active] Tarh [908529, ICBM Warpspawns] (prev: [12295363] Zzak'tarh)
4046041, exp:23, got:37

[12618231, Active] Gashokh [908529, ICBM Warpspawns] (prev: [12456141] Gashokh'vanderburth)
4055462, exp:22, got:28

[12618232, Active] Vanderburth [908529, ICBM Warpspawns] (prev: [12456141] Gashokh'vanderburth)
4055462, exp:22, got:28

